# Step1 Packaging data, clean the other nine regions, similar to Bolton

In [6]:
import pandas as pd

# dealing with nine regions
def process_epc_by_area(file_name, area_code):
    df = pd.read_csv(file_name)

    # convert datatime, remove missing date rows
    df["INSPECTION_DATE"] = pd.to_datetime(df["INSPECTION_DATE"], errors="coerce")
    df = df.dropna(subset=["INSPECTION_DATE"])
    # filter period
    df["year"] = df["INSPECTION_DATE"].dt.year
    df = df[(df["year"] >= 2013) & (df["year"] <= 2023)]
    # remove missing values
    df = df.dropna(subset=[
        "ENERGY_CONSUMPTION_CURRENT",
        "TOTAL_FLOOR_AREA",
        "MAIN_FUEL",
        "PROPERTY_TYPE",
        "BUILT_FORM",
        "BUILDING_REFERENCE_NUMBER"    
    ])
    # newest record of each building
    df = df.sort_values(by=["BUILDING_REFERENCE_NUMBER", "INSPECTION_DATE"])
    df = df.groupby(["BUILDING_REFERENCE_NUMBER", "year"]).tail(1).reset_index(drop=True)
    
    df['LOCAL_AUTHORITY'] = area_code
    return df

# Step2 nine regions call the fuction, mark lacode

In [8]:
area_code_map = {
    "Bury": "E08000002",
    "Manchester": "E08000003",
    "Oldham": "E08000004",
    "Rochdale": "E08000005",
    "Salford": "E08000006",
    "Stockport": "E08000007",
    "Tameside": "E08000008",
    "Trafford": "E08000009",
    "Wigan": "E08000010",
}

area_dfs = []
for area_name, area_code in area_code_map.items():
    filename = f"{area_name}.csv"
    print(f"Handling: {area_name} ({filename})")
    df = process_epc_by_area(filename, area_code)
    area_dfs.append(df)
    print(f"{area_name} done. {df.shape[0]} rows retained.")

Handling: Bury (Bury.csv)


C:\Users\chen\AppData\Local\Temp\ipykernel_19784\1303346672.py:5: DtypeWarning: Columns (34,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


Bury done. 54508 rows retained.
Handling: Manchester (Manchester.csv)


C:\Users\chen\AppData\Local\Temp\ipykernel_19784\1303346672.py:5: DtypeWarning: Columns (37,83) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


Manchester done. 192634 rows retained.
Handling: Oldham (Oldham.csv)


...

Stockport done. 77074 rows retained.
Handling: Tameside (Tameside.csv)


C:\Users\chen\AppData\Local\Temp\ipykernel_19784\1303346672.py:5: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


Tameside done. 70667 rows retained.
Handling: Trafford (Trafford.csv)
Trafford done. 61320 rows retained.
Handling: Wigan (Wigan.csv)


C:\Users\chen\AppData\Local\Temp\ipykernel_19784\1303346672.py:5: DtypeWarning: Columns (34,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


Wigan done. 86873 rows retained.


# Step3 Merge all 10 regions

In [11]:
# two datesets
combined_df = pd.concat(area_dfs, ignore_index=True)
bolton_df = pd.read_csv("cleaned_epc_certificates.csv", parse_dates=["INSPECTION_DATE"])

bolton_cols = bolton_df.columns

# remain cols Bolton has
combined_df = combined_df[bolton_cols]

# Check dtypes
print(bolton_df["INSPECTION_DATE"].dtype)
print(combined_df["INSPECTION_DATE"].dtype)

C:\Users\chen\AppData\Local\Temp\ipykernel_19784\2743022094.py:3: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  bolton_df = pd.read_csv("cleaned_epc_certificates.csv", parse_dates=["INSPECTION_DATE"])


datetime64[ns]
datetime64[ns]


In [13]:
# Merge
epc_all_df = pd.concat([bolton_df, combined_df], ignore_index=True)

# Checking
epc_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854276 entries, 0 to 854275
Data columns (total 84 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   LMK_KEY                        854276 non-null  object        
 1   ADDRESS1                       854276 non-null  object        
 2   ADDRESS2                       474554 non-null  object        
 3   POSTCODE                       854276 non-null  object        
 4   BUILDING_REFERENCE_NUMBER      854276 non-null  int64         
 5   CURRENT_ENERGY_RATING          854276 non-null  object        
 6   POTENTIAL_ENERGY_RATING        854276 non-null  object        
 7   CURRENT_ENERGY_EFFICIENCY      854276 non-null  int64         
 8   POTENTIAL_ENERGY_EFFICIENCY    854276 non-null  int64         
 9   PROPERTY_TYPE                  854276 non-null  object        
 10  BUILT_FORM                     854276 non-null  object        
 11  

# Step4 Sparsity analysis and labeling

In [16]:
import numpy as np
# thresholds
zero_threshold = 0.9
missing_threshold = 0.9
low_unique_threshold = 1

# 1.missing
missing_ratio = epc_all_df.isnull().mean() # 布尔值的mean就是率

# 2.zero
zero_ratio = epc_all_df.select_dtypes(include = [np.number]).apply(lambda x: (x == 0).mean())

# 3.unique counts
unique_counts = epc_all_df.nunique(dropna = False)

# 4.summarize
field_analysis_all = pd.DataFrame({
    'missing_ratio': missing_ratio,
    'zero_ratio': zero_ratio.reindex(epc_all_df.columns),
    'unique_count': unique_counts
}, index = epc_all_df.columns)

# 5.labeling
def classify_field(row):
    if row['missing_ratio'] > missing_threshold or row['zero_ratio'] > zero_threshold or row['unique_count'] <= low_unique_threshold:
        return 'drop_lstm'
    else:
        return 'keep_core'

field_analysis_all['keep_tag'] = field_analysis_all.apply(classify_field, axis = 1)

# define
drop_lstm_fields_all = field_analysis_all[field_analysis_all['keep_tag'] == 'drop_lstm'].index.tolist()

# results
def display_dataframe_to_user(name, dataframe):
    print(f"\n{name}\n{'=' * len(name)}")
    display(dataframe)
    
display_dataframe_to_user(name='Field sparsity and retention suggestions', dataframe=field_analysis_all)


Field sparsity and retention suggestions


,missing_ratio,zero_ratio,unique_count,keep_tag
LMK_KEY,0.000000,NaN,854276,keep_core
ADDRESS1,0.000000,NaN,658341,keep_core
ADDRESS2,0.444496,NaN,21523,keep_core
POSTCODE,0.000000,NaN,61196,keep_core
BUILDING_REFERENCE_NUMBER,0.000000,0.0,788699,keep_core
...,...,...,...,...
UPRN,0.009534,0.0,737422,keep_core
UPRN_SOURCE,0.009534,NaN,3,keep_core
REPORT_TYPE,0.000000,0.0,2,keep_core
year,0.000000,0.0,11,keep_core


In [18]:
# delete drop tags
epc_all_lstm_df = epc_all_df.drop(columns=drop_lstm_fields_all)
# remain local authority
epc_all_lstm_df['LOCAL_AUTHORITY'] = epc_all_df['LOCAL_AUTHORITY']

In [20]:
epc_all_df.to_csv("epc_all_cleaned.csv", index=False)

field_tags_all = field_analysis_all[['keep_tag']].copy()
field_tags_all.index.name = 'field'

field_tags_all.reset_index(inplace = True)

field_tags_all.to_csv('epc_all_field_tags.csv', index = False)